# Phase 1.1 — Universal Schema Alignment  `[v5.1 — 114-Dimensional Behavioral Schema]`

**Objective:** Transform the three v4 aligned Parquet files into the **Universal Behavioral Schema v5.1**
with protocol-triggered masking, 8-class UBT taxonomy, and evidence sensor logic.

**Output:** `aligned_schema_v5.parquet` — unified schema consumed by Phase 1.2.

---

## Universal Behavioral Schema v5.1 — 114 Dimensions

| Block | Dims | Contents |
|---|---|---|
| **Block 1 — Core** | 5 | duration, bytes_in, bytes_out, pkts_in, pkts_out |
| **Block 2 — Protocol** | 18 | proto OHE (6) + service OHE (12, gated by `has_svc`) |
| **Block 3 — State** | 5 | PENDING / ESTABLISHED / REJECTED / RESET / OTHER |
| **Block 4 — Port** | 16 | sport_func (7-OHE) + dport_func (7-OHE) + port_freq (2) |
| **Block 5 — App-Layer** | 51 | DNS (15) + HTTP (21) + SSL (15) |
| **Block 6 — Momentum** | 14 | 14 UNSW Bot-IoT window features |
| **Mask Bits** | 5 | has_svc, has_dns, has_http, has_ssl, has_unsw |
| **TOTAL** | **114** | |

**Masking Rule:** If mask bit = 0 → entire corresponding block forced to 0.0

## UBT Archetypes (8)
`NORMAL` · `SCAN` · `DOS_DDOS` · `BOTNET_C2` · `EXPLOIT` · `BRUTE_FORCE` · `THEFT_EXFIL` · `ANOMALY`


In [33]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 1 | Imports + Version Banner
# ─────────────────────────────────────────────────────────────────────────────
import sys, os, time, warnings
from pathlib import Path

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

SCHEMA_VERSION = "v5.1"
TOTAL_DIMS     = 114

# ── PyArrow (Parquet engine) ──────────────────────────────────────────────────
try:
    import pyarrow as pa
    import pyarrow.parquet as pq
    print(f"✅ pyarrow {pa.__version__}")
except ImportError:
    os.system(f"{sys.executable} -m pip install pyarrow -q")
    import pyarrow as pa
    import pyarrow.parquet as pq
    print(f"✅ pyarrow installed")

# ── tqdm (progress bars) ──────────────────────────────────────────────────────
try:
    from tqdm import tqdm
    print("✅ tqdm available")
except ImportError:
    def tqdm(it, **kw):
        return it
    print("⚠️  tqdm not found — plain iteration")

print(f"\n🔷 Universal Behavioral Schema {SCHEMA_VERSION} — {TOTAL_DIMS} Dimensions")
print(f"   Phase 1.1: Raw v4 Parquets → Aligned Schema v5.1")
print(f"\n✅ Python  {sys.version.split()[0]}")
print(f"✅ pandas  {pd.__version__}")
print(f"✅ numpy   {np.__version__}")


✅ pyarrow 23.0.0
✅ tqdm available

🔷 Universal Behavioral Schema v5.1 — 114 Dimensions
   Phase 1.1: Raw v4 Parquets → Aligned Schema v5.1

✅ Python  3.13.9
✅ pandas  2.2.3
✅ numpy   2.1.3


In [34]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 2 | Paths + Processing Constants
# ─────────────────────────────────────────────────────────────────────────────
NOTEBOOK_DIR  = Path.cwd()              # .../Phase_1/
MAIN_DIR      = NOTEBOOK_DIR.parent    # .../main_folder/
DATA_DIR      = MAIN_DIR / "data"
UNIFIED_DIR   = DATA_DIR / "unified"
ARTIFACTS_DIR = MAIN_DIR / "artifacts"
VECTORS_DIR   = DATA_DIR / "vectors"

# ── Raw Source Dataset Directories (Phase 0 original data — NOT v4 Parquets) ─
TONIOT_RAW_DIR  = DATA_DIR / "ton_iot"
IOT23_RAW_DIR   = DATA_DIR / "iot_23"
BOTIOT_RAW_DIR  = DATA_DIR / "bot_iot"

# ── Output paths ─────────────────────────────────────────────────────────────
ALIGNED_V51_PARQUET = UNIFIED_DIR / "aligned_schema_v5.parquet"  # legacy flat
OCEAN_V51_DIR       = UNIFIED_DIR / "ocean_v51"                   # Hive-partitioned IDS Ocean

# Streaming chunk size — 250K for high-speed throughput
CHUNK_SIZE = 250_000

# ── Create directories ────────────────────────────────────────────────────────
import shutil
for d in [UNIFIED_DIR, ARTIFACTS_DIR, VECTORS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# ── Discover ALL raw source files (no caps) ───────────────────────────────────
toniot_files = sorted(TONIOT_RAW_DIR.glob("Network_dataset_*.csv"))
botiot_files = sorted(BOTIOT_RAW_DIR.glob("UNSW_2018_IoT_Botnet_Full5pc_*.csv"))
iot23_logs   = sorted(IOT23_RAW_DIR.rglob("conn.log.labeled"))

print("=" * 68)
print(f"  Universal Behavioral Schema {SCHEMA_VERSION} — Phase 1.1  [IDS OCEAN LOADER]")
print(f"  Mode: FULL DATASET — NO ROW CAPS — Partitioned by ubt_archetype")
print("=" * 68)

print("\n📁 Raw data directories:")
for label, path in [
    ("TON-IoT raw",  TONIOT_RAW_DIR),
    ("IoT-23 raw",   IOT23_RAW_DIR),
    ("Bot-IoT raw",  BOTIOT_RAW_DIR),
]:
    status = "✅" if path.exists() else "❌ MISSING"
    print(f"  {status}  {label:<18}  {path}")

print("\n📂 Raw source file counts (ALL files — no caps):")
print(f"  TON-IoT CSV files   : {len(toniot_files):>4d}  (Network_dataset_*.csv)")
print(f"  IoT-23 log files    : {len(iot23_logs):>4d}  (conn.log.labeled)")
print(f"  Bot-IoT CSV files   : {len(botiot_files):>4d}  (UNSW_2018_IoT_Botnet_Full5pc_*.csv)")

print(f"\n📤 Ocean output : {OCEAN_V51_DIR}")
print(f"   Partition by: ubt_archetype  (8 Hive-style subdirectories)")
print(f"   Chunk size  : {CHUNK_SIZE:,} rows")

missing = []
if not toniot_files: missing.append("TON-IoT")
if not iot23_logs:   missing.append("IoT-23")

if not botiot_files: missing.append("Bot-IoT")

if missing:    raise FileNotFoundError(f"Raw source files missing for: {missing}. Verify DATA_DIR paths.")

  Universal Behavioral Schema v5.1 — Phase 1.1  [IDS OCEAN LOADER]
  Mode: FULL DATASET — NO ROW CAPS — Partitioned by ubt_archetype

📁 Raw data directories:
  ✅  TON-IoT raw         c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\data\ton_iot
  ✅  IoT-23 raw          c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\data\iot_23
  ✅  Bot-IoT raw         c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\data\bot_iot

📂 Raw source file counts (ALL files — no caps):
  TON-IoT CSV files   :   23  (Network_dataset_*.csv)
  IoT-23 log files    :   23  (conn.log.labeled)
  Bot-IoT CSV files   :    4  (UNSW_2018_IoT_Botnet_Full5pc_*.csv)

📤 Ocean output : c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\data\unified\ocean_v51
   Partition by: ubt_archetype  (8 Hive-style subdirectories)

---
## Section 1 — Schema Registries

UBT Taxonomy (8 archetypes), State 5-token OHE, Protocol/Service vocabularies, Port Function 7-way classifier.
All lookup tables are closed and authoritative — no magic strings.


In [35]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 3 | Schema Registries — UBT Taxonomy + State / Protocol / Port /
#          HTTP Method / SSL Cipher (raw DNA from source files)
# ─────────────────────────────────────────────────────────────────────────────

# ── 3A: UBT Archetype Taxonomy (8 classes) ───────────────────────────────────
UBT_ARCHETYPES = [
    "NORMAL", "SCAN", "DOS_DDOS", "BOTNET_C2",
    "EXPLOIT", "BRUTE_FORCE", "THEFT_EXFIL", "ANOMALY"
]

# ── Zero-Loss Multi-Tier Inclusive Substring Taxonomy ────────────────────────
# Priority order matters: first match wins (most specific first).
# Case-insensitive substring search: if ANY keyword is in label.lower() → match.
UBT_PRIORITY_TIERS = [
    # Tier 1 — BOTNET_C2 (C&C infrastructure, IoT botnets)
    ("BOTNET_C2", [
        "mirai", "hajime", "muhstik", "kenjiro", "okiru", "torii",
        "linux.mirai", "cnc", "c&c", "heartbeat", "viriut",
        "hide_and_seek", "trojan", "backdoor", "commandexecution",
        "gafgyt", "bashlite", "lightaidra", "aidra",
    ]),
    # Tier 2 — BRUTE_FORCE (credential attacks) — placed BEFORE EXPLOIT to avoid
    #           'bruteforce' matching 'rce' substring in EXPLOIT tier
    ("BRUTE_FORCE", [
        "brute", "password", "bruteforce", "credential", "guessing",
        "dictionary", "hydra", "medusa",
    ]),
    # Tier 3 — EXPLOIT (code execution, injection, memory corruption)
    ("EXPLOIT", [
        "injection", "xss", "exploit", "ransomware", "heartbleed",
        "shellshock", "uaf", "poisoning", "mitm", "overflow",
        "remote_code", "eternalblue", "bluekeep", "log4j", "apachestruts",
    ]),
    # Tier 4 — THEFT_EXFIL (data theft, credential leak, spyware)
    ("THEFT_EXFIL", [
        "theft", "exfiltration", "malware_download", "spyware",
        "keylogger", "keylogging", "creds", "stealing", "leak",
        "filedownload", "data_exfiltration",
    ]),
    # Tier 5 — SCAN (reconnaissance, port scanning)
    ("SCAN", [
        "partofahorizontalportscan",           # IoT-23 'label' col — rescue 213M rows
        "horizontalportscan", "portscan",      # generic variants
        "scanning", "os_fingerprint", "recon",
        "discovery", "reconnaissance", "service_scan",
        "fingerprint", "sweep", "probing",
    ]),
    # Tier 6 — DOS_DDOS (denial of service, flooding)
    ("DOS_DDOS", [
        "dos", "ddos", "flood", "smurf", "land", "teardrop", "attack",
        "slowloris", "goldeneye", "rudy", "syn", "ack", "rst",
        "amplif", "reflection",
    ]),
    # Tier 7 — NORMAL (benign baseline)
    ("NORMAL", [
        "normal", "benign", "legitimate", "background", "unknown",
    ]),
]

# Flat legacy map kept for backward compat with any existing callers
UBT_KEYWORD_MAP = {kw: arch for arch, kws in UBT_PRIORITY_TIERS for kw in kws}

V4_CLASS_TO_UBT = {0: "NORMAL", 1: "SCAN", 2: "DOS_DDOS",
                   3: "BOTNET_C2", 4: "EXPLOIT", -1: "ANOMALY"}

# Pre-compile tier tuple for fast iteration
_UBT_TIERS_COMPILED = [
    (arch, tuple(kws)) for arch, kws in UBT_PRIORITY_TIERS
]

def map_to_ubt(specific_attack: str, multiclass_int=None) -> str:
    """
    Zero-loss inclusive UBT mapping.
    Uses multi-tier case-insensitive SUBSTRING search (priority order).
    Preserves raw label in univ_specific_attack (caller's responsibility).
    Falls back to multiclass int if provided, then ANOMALY.
    """
    label = str(specific_attack).lower().strip()
    # Remove leading/trailing punctuation common in IoT-23 labels
    label = label.lstrip("-").strip()

    # Exact sentinel: empty / truly missing → ANOMALY
    if not label or label in {"-", "?", "(empty)", "nan", "none"}:
        if multiclass_int is not None:
            try:
                mc = int(float(multiclass_int))
            except (ValueError, TypeError):
                mc = -1
            return V4_CLASS_TO_UBT.get(mc, "ANOMALY")
        return "ANOMALY"

    # Multi-tier substring scan (first match wins)
    for arch, keywords in _UBT_TIERS_COMPILED:
        if any(kw in label for kw in keywords):
            return arch

    # Integer-class fallback
    if multiclass_int is not None:
        try:
            mc = int(float(multiclass_int))
        except (ValueError, TypeError):
            mc = -1
        return V4_CLASS_TO_UBT.get(mc, "ANOMALY")

    return "ANOMALY"

# ── 3B: State mapping → v5.1 5-token OHE ─────────────────────────────────────
STATE_V51_MAP = {
    # Zeek conn_state codes
    "S0":      "PENDING",      "S1":      "PENDING",
    "OTH":     "PENDING",      "REQ":     "PENDING",
    "SF":      "ESTABLISHED",  "S2":      "ESTABLISHED",
    "S3":      "ESTABLISHED",  "CON":     "ESTABLISHED",
    "FIN":     "ESTABLISHED",
    "REJ":     "REJECTED",
    "RSTO":    "RESET",        "RSTOS0":  "RESET",
    "RSTR":    "RESET",        "RSTRH":   "RESET",
    "RST":     "RESET",
    "SHR":     "OTHER",        "SH":      "OTHER",
    "INT":     "OTHER",        "URN":     "OTHER",
    "ECO":     "OTHER",        "ECR":     "OTHER",
    "-":       "OTHER",        "?":       "OTHER",
    "(empty)": "OTHER",        "":        "OTHER",
    # Argus (Bot-IoT)
    "ACC":     "ESTABLISHED",  "CLO":     "ESTABLISHED",
}
UBT_STATE_TOKENS = ["PENDING", "ESTABLISHED", "REJECTED", "RESET", "OTHER"]

# ── 3C: Protocol OHE vocabulary (6 dims) ──────────────────────────────────────
PROTO_TOKENS = ["tcp", "udp", "icmp", "arp", "ipv6", "other"]
PROTO_IDX    = {t: i for i, t in enumerate(PROTO_TOKENS)}

# ── 3D: Service OHE vocabulary (12 dims, gated by has_svc) ───────────────────
SERVICE_TOKENS = [
    "dns", "http", "ssl", "ftp", "ssh", "smtp",
    "dhcp", "quic", "ntp", "rdp", "pop3", "other"
]
SERVICE_IDX  = {t: i for i, t in enumerate(SERVICE_TOKENS)}
SERVICE_SET  = set(SERVICE_TOKENS)
ABSENT_SERVICE_VALS = frozenset({"<absent>", "-", "unknown", "", "none", "(empty)"})

# ── 3E: Port Function 7-way OHE ───────────────────────────────────────────────
PORT_FUNC_TOKENS = [
    "SCADA_CONTROL", "IOT_MANAGEMENT", "WEB_SERVICES",
    "NETWORK_CORE", "REMOTE_ACCESS", "FUNC_EPHEMERAL", "FUNC_UNKNOWN"
]
SCADA_PORTS    = frozenset({502, 102, 44818})
IOT_MGMT_PORTS = frozenset({1883, 5683, 8883})
WEB_PORTS      = frozenset({80, 443, 8080})
NET_CORE_PORTS = frozenset({53, 67, 68, 123})
REMOTE_PORTS   = frozenset({22, 23, 3389})

def classify_port(port) -> str:
    try:
        p = int(float(port))
    except (ValueError, TypeError):
        return "FUNC_UNKNOWN"
    if p < 0:               return "FUNC_UNKNOWN"
    if p in SCADA_PORTS:    return "SCADA_CONTROL"
    if p in IOT_MGMT_PORTS: return "IOT_MANAGEMENT"
    if p in WEB_PORTS:      return "WEB_SERVICES"
    if p in NET_CORE_PORTS: return "NETWORK_CORE"
    if p in REMOTE_PORTS:   return "REMOTE_ACCESS"
    if p > 49152:           return "FUNC_EPHEMERAL"
    return "FUNC_UNKNOWN"

# ── 3F: HTTP Method OHE (8 dims) — raw DNA from TON-IoT ─────────────────────
HTTP_METHOD_TOKENS = ["GET", "POST", "PUT", "DELETE", "HEAD", "OPTIONS", "PATCH", "OTHER"]
HTTP_METHOD_IDX    = {t: i for i, t in enumerate(HTTP_METHOD_TOKENS)}

# ── 3G: SSL Cipher OHE (12 dims) — raw DNA from TON-IoT ssl_cipher ───────────
# Top 11 cipher suites observed in IoT/industrial traffic + "other"
SSL_CIPHER_TOKENS = [
    "TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256",          # 0
    "TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384",          # 1
    "TLS_RSA_WITH_AES_128_GCM_SHA256",                # 2
    "TLS_RSA_WITH_AES_256_GCM_SHA384",                # 3
    "TLS_RSA_WITH_AES_128_CBC_SHA",                   # 4
    "TLS_RSA_WITH_AES_256_CBC_SHA",                   # 5
    "TLS_RSA_WITH_RC4_128_SHA",                       # 6 — weak (IoT legacy)
    "TLS_RSA_WITH_RC4_128_MD5",                       # 7 — weak
    "TLS_RSA_WITH_3DES_EDE_CBC_SHA",                  # 8 — legacy
    "TLS_DHE_RSA_WITH_AES_128_CBC_SHA",               # 9
    "TLS_ECDHE_ECDSA_WITH_AES_128_GCM_SHA256",        # 10
    "other",                                          # 11
]
SSL_CIPHER_IDX = {t: i for i, t in enumerate(SSL_CIPHER_TOKENS)}

# ── 3H: SSL Version OHE (3 dims: weak/strong/absent) ─────────────────────────
# Appended after cipher OHE to complete 15 dims for Block 5 SSL
#   Dim 12: weak  (SSLv2, SSLv3, TLSv1.0, TLSv1.1)
#   Dim 13: strong (TLSv1.2, TLSv1.3)
#   Dim 14: ssl_established flag (0/1)
WEAK_SSL_VERSIONS   = frozenset({"sslv2","sslv3","tlsv1","tlsv10","tlsv1.0","tls1.0"})
STRONG_SSL_VERSIONS = frozenset({"tlsv12","tlsv13","tlsv1.2","tlsv1.3","tls1.2","tls1.3"})

# ── 3I: v5.1 Aligned Schema Output Columns ────────────────────────────────────
V51_OUTPUT_COLUMNS = [
    # Identity + Metadata
    "dataset_source", "meta_src_ip", "meta_dst_ip", "meta_timestamp",
    # Labels
    "ubt_archetype", "univ_specific_attack",
    # Block 1: Core
    "univ_duration", "univ_bytes_in", "univ_bytes_out", "univ_pkts_in", "univ_pkts_out",
    # Block 2: Protocol
    "raw_proto", "raw_service",
    # Block 3: State
    "raw_state_v51",
    # Block 4: Port
    "raw_sport", "raw_dport",
    # Block 5: DNS (raw codes)
    "dns_qtype", "dns_qclass", "dns_rcode",
    # Block 5: HTTP (raw method + status + body — ACTUAL from source, not proxy)
    "raw_http_method", "http_status_code", "http_req_body_len", "http_resp_body_len",
    # Block 5: SSL (raw cipher + version + established — ACTUAL from source)
    "raw_ssl_cipher", "raw_ssl_version", "ssl_established",
    # Block 6: Momentum (14 UNSW window features, -1.0 when absent)
    "mom_mean", "mom_stddev", "mom_sum", "mom_min", "mom_max",
    "mom_rate", "mom_srate", "mom_drate",
    "mom_TnBPSrcIP", "mom_TnBPDstIP",
    "mom_TnP_PSrcIP", "mom_TnP_PDstIP",
    "mom_TnP_PerProto", "mom_TnP_Per_Dport",
    # Mask Bits
    "has_svc", "has_dns", "has_http", "has_ssl", "has_unsw",
]

print(f"✅ UBT Archetypes    ({len(UBT_ARCHETYPES):2d}): {', '.join(UBT_ARCHETYPES)}")
print(f"✅ State tokens      ({len(UBT_STATE_TOKENS):2d}): {', '.join(UBT_STATE_TOKENS)}")
print(f"✅ Proto tokens      ({len(PROTO_TOKENS):2d}): {', '.join(PROTO_TOKENS)}")
print(f"✅ Service tokens    ({len(SERVICE_TOKENS):2d}): {', '.join(SERVICE_TOKENS)}")
print(f"✅ Port functions    ({len(PORT_FUNC_TOKENS):2d}): {', '.join(PORT_FUNC_TOKENS)}")
print(f"✅ HTTP Method OHE   ({len(HTTP_METHOD_TOKENS):2d}): {', '.join(HTTP_METHOD_TOKENS)}")
print(f"✅ SSL Cipher OHE    ({len(SSL_CIPHER_TOKENS):2d}): top-11 ciphers + other")
print(f"✅ SSL Version dims  ( 3): weak / strong / ssl_established")
print(f"✅ Output columns    ({len(V51_OUTPUT_COLUMNS):2d}): v5.1 aligned schema")
total_keywords = sum(len(kws) for _, kws in UBT_PRIORITY_TIERS)
print(f"\n🎯 Zero-Loss Taxonomy (multi-tier inclusive substring search):")
print(f"   Strategy  : case-insensitive substring scan, priority-ordered tiers")
print(f"   Tiers     : {len(UBT_PRIORITY_TIERS)}  (BOTNET_C2 → EXPLOIT → THEFT_EXFIL → BRUTE_FORCE → SCAN → DOS_DDOS → NORMAL)")
print(f"   Keywords  : {total_keywords} total across all tiers")
for arch, kws in UBT_PRIORITY_TIERS:
    print(f"   {arch:<14}: {len(kws):2d} keywords")
print(f"\n   Smoke-test (10 cases — incl. IoT-23 primary labels):")
for lbl, expected in [
    ("c&c-heartbeat-attack",      "BOTNET_C2"),
    ("PartOfAHorizontalPortScan", "SCAN"),        # IoT-23 'label' col — 213M rows
    ("PartOfHorizontalPortScan",  "SCAN"),
    ("DDoS",                      "DOS_DDOS"),
    ("BruteForce",                "BRUTE_FORCE"),
    ("FileDownload",              "THEFT_EXFIL"),
    ("Okiru",                     "BOTNET_C2"),
    ("normal",                    "NORMAL"),
    ("benign",                    "NORMAL"),
    ("-",                         "ANOMALY"),
]:
    got = map_to_ubt(lbl)
    flag = "✅" if got == expected else f"❌ got {got}"
    print(f"   {flag}  '{lbl}' → {expected}")
print(f"\n🔑 KEY BREAKTHROUGH:")
print(f"   raw_http_method  → actual GET/POST/DELETE etc. from TON-IoT source")
print(f"   raw_ssl_cipher   → actual TLS cipher suite string from TON-IoT source")
print(f"   raw_ssl_version  → actual TLSv1.2 / TLSv1.3 etc. from TON-IoT source")
print(f"   ssl_established  → actual handshake outcome (0/1) from TON-IoT source")

✅ UBT Archetypes    ( 8): NORMAL, SCAN, DOS_DDOS, BOTNET_C2, EXPLOIT, BRUTE_FORCE, THEFT_EXFIL, ANOMALY
✅ State tokens      ( 5): PENDING, ESTABLISHED, REJECTED, RESET, OTHER
✅ Proto tokens      ( 6): tcp, udp, icmp, arp, ipv6, other
✅ Service tokens    (12): dns, http, ssl, ftp, ssh, smtp, dhcp, quic, ntp, rdp, pop3, other
✅ Port functions    ( 7): SCADA_CONTROL, IOT_MANAGEMENT, WEB_SERVICES, NETWORK_CORE, REMOTE_ACCESS, FUNC_EPHEMERAL, FUNC_UNKNOWN
✅ HTTP Method OHE   ( 8): GET, POST, PUT, DELETE, HEAD, OPTIONS, PATCH, OTHER
✅ SSL Cipher OHE    (12): top-11 ciphers + other
✅ SSL Version dims  ( 3): weak / strong / ssl_established
✅ Output columns    (45): v5.1 aligned schema

🎯 Zero-Loss Taxonomy (multi-tier inclusive substring search):
   Strategy  : case-insensitive substring scan, priority-ordered tiers
   Tiers     : 7  (BOTNET_C2 → EXPLOIT → THEFT_EXFIL → BRUTE_FORCE → SCAN → DOS_DDOS → NORMAL)
   Keywords  : 85 total across all tiers
   BOTNET_C2     : 19 keywords
   BRUTE_FORC

---
## Section 2 — Evidence Sensor Logic (5 Mask Bits)

Mask bits are derived from feature existence, **not** dataset source.
Each bit gates an entire block — if bit = 0, the block is zeroed in Phase 1.2.


In [36]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 4 | Evidence Sensor Logic — 5 Mask Bits (using Raw Column Names)
# ─────────────────────────────────────────────────────────────────────────────
#
# Mask bits are derived from feature EXISTENCE in the RAW source,
# not inferred from downstream proxies.
#
# Raw column names (as they appear in source CSVs/logs):
#   TON-IoT  : service, dns_qtype, http_method, http_status_code,
#              ssl_cipher, ssl_version, ssl_established
#   IoT-23   : service  (conn.log.labeled — no app-layer columns)
#   Bot-IoT  : proto, state, TnBPSrcIP (14 window cols)
#
# ── Sentinel values used in raw files ────────────────────────────────────────
RAW_ABSENT = frozenset({"-", "(empty)", "", "none", "nan", "<absent>"})

HTTP_DETECT_DPORTS = frozenset({80, 443, 8080})


def _raw_is_absent(val) -> bool:
    """Return True if a raw string value is a missing/sentinel value."""
    return str(val).strip().lower() in RAW_ABSENT


def compute_mask_bits_raw(
    service_col,      # pd.Series: raw 'service' column
    dns_qtype_col,    # pd.Series or None
    http_method_col,  # pd.Series or None
    http_status_col,  # pd.Series or None
    ssl_cipher_col,   # pd.Series or None
    ssl_version_col,  # pd.Series or None
    dport_col,        # pd.Series or None
    unsw_col,         # pd.Series or None  (TnBPSrcIP)
    n: int,
    index,
) -> dict:
    """
    Compute all 5 mask bits from raw source column Series.
    Returns dict of 5 int8 Series.
    """
    svc = service_col.fillna("-").astype(str).str.lower().str.strip()

    # ── has_svc ───────────────────────────────────────────────────────────────
    has_svc = (~svc.isin(RAW_ABSENT)).astype("int8")

    # ── has_dns ───────────────────────────────────────────────────────────────
    dns_from_svc = (svc == "dns")
    if dns_qtype_col is not None:
        dns_from_qtype = ~dns_qtype_col.fillna("-").astype(str).str.strip().str.lower().isin(RAW_ABSENT)
    else:
        dns_from_qtype = pd.Series(False, index=index)
    has_dns = (dns_from_svc | dns_from_qtype).astype("int8")

    # ── has_http ──────────────────────────────────────────────────────────────
    http_from_svc  = svc.isin({"http", "ssl"})  # ssl port 443 is often HTTP/2
    if http_method_col is not None:
        http_from_method = ~http_method_col.fillna("-").astype(str).str.strip().str.lower().isin(RAW_ABSENT)
    else:
        http_from_method = pd.Series(False, index=index)
    if http_status_col is not None:
        http_from_status = ~http_status_col.fillna("-").astype(str).str.strip().str.lower().isin(RAW_ABSENT)
    else:
        http_from_status = pd.Series(False, index=index)
    if dport_col is not None:
        dport_num        = pd.to_numeric(dport_col, errors="coerce").fillna(-1).astype(int)
        http_from_dport  = dport_num.isin(HTTP_DETECT_DPORTS)
    else:
        http_from_dport  = pd.Series(False, index=index)
    has_http = (http_from_svc | http_from_method | http_from_status | http_from_dport).astype("int8")

    # ── has_ssl ───────────────────────────────────────────────────────────────
    ssl_from_svc = (svc == "ssl")
    if ssl_cipher_col is not None:
        ssl_from_cipher = ~ssl_cipher_col.fillna("-").astype(str).str.strip().str.lower().isin(RAW_ABSENT)
    else:
        ssl_from_cipher = pd.Series(False, index=index)
    if ssl_version_col is not None:
        ssl_from_version = ~ssl_version_col.fillna("-").astype(str).str.strip().str.lower().isin(RAW_ABSENT)
    else:
        ssl_from_version = pd.Series(False, index=index)
    has_ssl = (ssl_from_svc | ssl_from_cipher | ssl_from_version).astype("int8")

    # ── has_unsw ──────────────────────────────────────────────────────────────
    if unsw_col is not None:
        has_unsw = (~unsw_col.fillna(-1).astype(str).str.strip().isin(RAW_ABSENT) & (
            pd.to_numeric(unsw_col, errors="coerce").fillna(-1) >= 0
        )).astype("int8")
    else:
        has_unsw = pd.Series(np.int8(0), index=index)

    return {
        "has_svc":  has_svc,
        "has_dns":  has_dns,
        "has_http": has_http,
        "has_ssl":  has_ssl,
        "has_unsw": has_unsw,
    }


print("✅ Evidence sensor (raw columns) defined")
print("   has_svc  : service ∉ {-, (empty), ''}")
print("   has_dns  : service=='dns'  OR  dns_qtype is not missing")
print("   has_http : service∈{http,ssl}  OR  http_method present  OR  dport∈{80,443,8080}")
print("             OR  http_status_code present")
print("   has_ssl  : service=='ssl'  OR  ssl_cipher present  OR  ssl_version present")
print("   has_unsw : TnBPSrcIP column present AND >= 0")
print()
print("🔑 KEY: has_ssl and has_http are now based on ACTUAL cipher/method columns")
print("        NOT on port/state proxies — full Resolution restored")


✅ Evidence sensor (raw columns) defined
   has_svc  : service ∉ {-, (empty), ''}
   has_dns  : service=='dns'  OR  dns_qtype is not missing
   has_http : service∈{http,ssl}  OR  http_method present  OR  dport∈{80,443,8080}
             OR  http_status_code present
   has_ssl  : service=='ssl'  OR  ssl_cipher present  OR  ssl_version present
   has_unsw : TnBPSrcIP column present AND >= 0

🔑 KEY: has_ssl and has_http are now based on ACTUAL cipher/method columns
        NOT on port/state proxies — full Resolution restored


---
## Section 3 — Dataset-Specific Chunk Transformers

Each `transform_*_chunk()` function maps one raw source chunk → v5.1 aligned schema row-set.
All 45 columns in `V51_OUTPUT_COLUMNS` are produced with correct dtypes.
Every port is classified by the **7-way Function Map** via `classify_port()`.

Every label is resolved to one of the **8 UBT archetypes** via `map_to_ubt()`.

In [38]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 5 | Dataset-Specific Chunk Transformers
#          Each function accepts a raw chunk → returns V51_OUTPUT_COLUMNS
# ─────────────────────────────────────────────────────────────────────────────

def _safe_num(series, fill=0.0) -> pd.Series:
    return pd.to_numeric(series, errors="coerce").fillna(fill)

def _clean_str(val, fallback="") -> str:
    s = str(val).strip()
    return fallback if s.lower() in RAW_ABSENT else s

def _build_output_frame(index) -> pd.DataFrame:
    """Return empty DataFrame with V51_OUTPUT_COLUMNS, correct types."""
    return pd.DataFrame(index=index)


# ─────────────────────────────────────────────────────────────────────────────
# TON-IoT Transform
# Source columns: ts, src_ip, src_port, dst_ip, dst_port, proto, service,
#   duration, src_bytes, dst_bytes, conn_state, missed_bytes, src_pkts,
#   dst_pkts, src_ip_bytes, dst_ip_bytes,
#   dns_query, dns_qclass, dns_qtype, dns_rcode,
#   ssl_version, ssl_cipher, ssl_resumed, ssl_established, ssl_subject, ssl_issuer,
#   http_trans_depth, http_method, http_uri, http_referrer, http_version,
#   http_request_body_len, http_response_body_len, http_status_code,
#   http_user_agent, http_orig_mime_types, http_resp_mime_types,
#   weird_name, weird_addl, weird_notice,
#   label, type
# ─────────────────────────────────────────────────────────────────────────────
def transform_toniot_chunk(chunk: pd.DataFrame) -> pd.DataFrame:
    """TON-IoT CSV chunk → v5.1 aligned schema (full HTTP/SSL/DNS resolution)."""
    n   = len(chunk)
    idx = chunk.index
    out = _build_output_frame(idx)

    # ── Identity / Metadata ───────────────────────────────────────────────────
    out["dataset_source"]     = "toniot"
    out["meta_src_ip"]        = chunk.get("src_ip",  pd.Series([""] * n, index=idx)).fillna("").astype(str)
    out["meta_dst_ip"]        = chunk.get("dst_ip",  pd.Series([""] * n, index=idx)).fillna("").astype(str)
    out["meta_timestamp"]     = _safe_num(chunk.get("ts", pd.Series(0.0, index=idx)), fill=0.0)

    # ── UBT label from 'type' (attack subtype) ────────────────────────────────
    attack_type = chunk.get("type", pd.Series(["normal"] * n, index=idx)).fillna("normal").astype(str)
    label_bin   = chunk.get("label", pd.Series([0] * n, index=idx)).fillna(0)
    # If label==0 (binary normal), force NORMAL regardless of type
    ubt_list = []
    for raw_type, lb in zip(attack_type, label_bin):
        if str(lb).strip() in {"0", "normal", "Normal"}:
            ubt_list.append("NORMAL")
        else:
            ubt_list.append(map_to_ubt(raw_type))
    out["ubt_archetype"]      = ubt_list
    out["univ_specific_attack"] = attack_type.astype(str)

    # ── Block 1: Core ─────────────────────────────────────────────────────────
    out["univ_duration"]  = _safe_num(chunk.get("duration",  pd.Series(0.0, index=idx))).clip(lower=0)
    out["univ_bytes_in"]  = _safe_num(chunk.get("src_bytes", pd.Series(0.0, index=idx))).clip(lower=0)
    out["univ_bytes_out"] = _safe_num(chunk.get("dst_bytes", pd.Series(0.0, index=idx))).clip(lower=0)
    out["univ_pkts_in"]   = _safe_num(chunk.get("src_pkts",  pd.Series(0.0, index=idx))).clip(lower=0)
    out["univ_pkts_out"]  = _safe_num(chunk.get("dst_pkts",  pd.Series(0.0, index=idx))).clip(lower=0)

    # ── Block 2: Protocol ─────────────────────────────────────────────────────
    valid_protos = frozenset(PROTO_TOKENS) - {"other"}
    raw_proto = chunk.get("proto", pd.Series(["other"] * n, index=idx)).fillna("other").astype(str).str.lower().str.strip()
    out["raw_proto"]   = raw_proto.apply(lambda p: p if p in valid_protos else "other")
    raw_svc   = chunk.get("service", pd.Series(["-"] * n, index=idx)).fillna("-").astype(str).str.lower().str.strip()
    out["raw_service"] = raw_svc.apply(
        lambda s: (s if s not in RAW_ABSENT and s in SERVICE_SET else
                   ("other" if s not in RAW_ABSENT else "<absent>"))
    )

    # ── Block 3: State ────────────────────────────────────────────────────────
    conn_state = chunk.get("conn_state", pd.Series(["-"] * n, index=idx)).fillna("-").astype(str).str.strip()
    out["raw_state_v51"] = conn_state.map(STATE_V51_MAP).fillna("OTHER")

    # ── Block 4: Ports ────────────────────────────────────────────────────────
    out["raw_sport"] = _safe_num(chunk.get("src_port", pd.Series(-1, index=idx)), fill=-1).astype(int)
    out["raw_dport"] = _safe_num(chunk.get("dst_port", pd.Series(-1, index=idx)), fill=-1).astype(int)

    # ── Block 5: DNS — ACTUAL FROM SOURCE ────────────────────────────────────
    out["dns_qtype"]  = _safe_num(chunk.get("dns_qtype",  pd.Series(-1, index=idx)), fill=-1).astype(int)
    out["dns_qclass"] = _safe_num(chunk.get("dns_qclass", pd.Series(-1, index=idx)), fill=-1).astype(int)
    out["dns_rcode"]  = _safe_num(chunk.get("dns_rcode",  pd.Series(-1, index=idx)), fill=-1).astype(int)

    # ── Block 5: HTTP — ACTUAL FROM SOURCE (raw http_method preserved) ────────
    raw_http_method = chunk.get("http_method", pd.Series(["-"] * n, index=idx)).fillna("-").astype(str).str.strip().str.upper()
    out["raw_http_method"]    = raw_http_method.apply(lambda m: m if m not in {"-","", "(EMPTY)", "NAN"} else "-")
    out["http_status_code"]   = _safe_num(chunk.get("http_status_code",        pd.Series(-1, index=idx)), fill=-1).astype(int)
    out["http_req_body_len"]  = _safe_num(chunk.get("http_request_body_len",   pd.Series(-1, index=idx)), fill=-1).astype(int)
    out["http_resp_body_len"] = _safe_num(chunk.get("http_response_body_len",  pd.Series(-1, index=idx)), fill=-1).astype(int)

    # ── Block 5: SSL — ACTUAL FROM SOURCE (raw ssl_cipher, ssl_version) ───────
    raw_ssl_cipher  = chunk.get("ssl_cipher",      pd.Series(["-"] * n, index=idx)).fillna("-").astype(str).str.strip()
    raw_ssl_version = chunk.get("ssl_version",     pd.Series(["-"] * n, index=idx)).fillna("-").astype(str).str.strip()
    ssl_estab_raw   = chunk.get("ssl_established", pd.Series(["-"] * n, index=idx)).fillna("-").astype(str).str.lower().str.strip()
    out["raw_ssl_cipher"]  = raw_ssl_cipher
    out["raw_ssl_version"] = raw_ssl_version
    out["ssl_established"] = ssl_estab_raw.map({"t": 1, "true": 1, "1": 1, "f": 0, "false": 0, "0": 0}).fillna(0).astype("int8")

    # ── Block 6: Momentum — all absent for TON-IoT ────────────────────────────
    mom_cols = ["mom_mean","mom_stddev","mom_sum","mom_min","mom_max","mom_rate",
                "mom_srate","mom_drate","mom_TnBPSrcIP","mom_TnBPDstIP",
                "mom_TnP_PSrcIP","mom_TnP_PDstIP","mom_TnP_PerProto","mom_TnP_Per_Dport"]
    for col in mom_cols:
        out[col] = -1.0

    # ── Mask bits (using raw columns) ─────────────────────────────────────────
    masks = compute_mask_bits_raw(
        service_col     = chunk.get("service",         pd.Series(["-"] * n, index=idx)),
        dns_qtype_col   = chunk.get("dns_qtype",       None),
        http_method_col = chunk.get("http_method",     None),
        http_status_col = chunk.get("http_status_code",None),
        ssl_cipher_col  = chunk.get("ssl_cipher",      None),
        ssl_version_col = chunk.get("ssl_version",     None),
        dport_col       = chunk.get("dst_port",        None),
        unsw_col        = None,
        n=n, index=idx,
    )
    for k, v in masks.items():
        out[k] = v

    return out[V51_OUTPUT_COLUMNS]


# ─────────────────────────────────────────────────────────────────────────────
# IoT-23 Transform
# Source columns: ts, uid, id.orig_h, id.orig_p, id.resp_h, id.resp_p,
#   proto, service, duration, orig_bytes, resp_bytes, conn_state,
#   local_orig, local_resp, missed_bytes, history, orig_pkts, orig_ip_bytes,
#   resp_pkts, resp_ip_bytes, tunnel_parents, label, detailed-label
# ─────────────────────────────────────────────────────────────────────────────
def transform_iot23_chunk(chunk: pd.DataFrame) -> pd.DataFrame:
    """IoT-23 Zeek conn.log chunk → v5.1 aligned schema."""
    n   = len(chunk)
    idx = chunk.index
    out = _build_output_frame(idx)

    # ── Identity / Metadata ───────────────────────────────────────────────────
    out["dataset_source"] = "iot23"
    out["meta_src_ip"]    = chunk.get("id.orig_h", pd.Series([""] * n, index=idx)).fillna("").astype(str)
    out["meta_dst_ip"]    = chunk.get("id.resp_h", pd.Series([""] * n, index=idx)).fillna("").astype(str)
    out["meta_timestamp"] = _safe_num(chunk.get("ts", pd.Series(0.0, index=idx)), fill=0.0)

    # ── UBT label: prefer 'detailed-label'; fall back to 'label' ─────────────
    # IoT-23 stores broad tags in 'label' (e.g. 'PartOfAHorizontalPortScan',
    # 'Benign') and specifics in 'detailed-label' (e.g. 'C&C-HeartBeat',
    # 'Okiru'). 'detailed-label' is '-' for most benign/scan rows → we MUST
    # fall back to 'label' to avoid dumping 213M scan rows into ANOMALY.
    det_label  = chunk.get("detailed-label", pd.Series(["-"] * n, index=idx)).fillna("-").astype(str).str.strip()
    label_col  = chunk.get("label",          pd.Series(["-"] * n, index=idx)).fillna("-").astype(str).str.strip()
    _SENT = frozenset({"-", "?", "(empty)", "nan", "", "none"})
    combined = [
        dl if dl.lower().lstrip("-").strip() not in _SENT else lc
        for dl, lc in zip(det_label, label_col)
    ]
    ubt_list = [map_to_ubt(c) for c in combined]
    out["ubt_archetype"]        = ubt_list
    out["univ_specific_attack"] = pd.Series(combined, index=idx)

    # ── Block 1: Core ─────────────────────────────────────────────────────────
    out["univ_duration"]  = _safe_num(chunk.get("duration",   pd.Series(0.0, index=idx))).clip(lower=0)
    out["univ_bytes_in"]  = _safe_num(chunk.get("orig_bytes", pd.Series(0.0, index=idx))).clip(lower=0)
    out["univ_bytes_out"] = _safe_num(chunk.get("resp_bytes", pd.Series(0.0, index=idx))).clip(lower=0)
    out["univ_pkts_in"]   = _safe_num(chunk.get("orig_pkts",  pd.Series(0.0, index=idx))).clip(lower=0)
    out["univ_pkts_out"]  = _safe_num(chunk.get("resp_pkts",  pd.Series(0.0, index=idx))).clip(lower=0)

    # ── Block 2: Protocol ─────────────────────────────────────────────────────
    valid_protos = frozenset(PROTO_TOKENS) - {"other"}
    raw_proto = chunk.get("proto", pd.Series(["other"] * n, index=idx)).fillna("other").astype(str).str.lower().str.strip()
    out["raw_proto"] = raw_proto.apply(lambda p: p if p in valid_protos else "other")
    raw_svc = chunk.get("service", pd.Series(["-"] * n, index=idx)).fillna("-").astype(str).str.lower().str.strip()
    out["raw_service"] = raw_svc.apply(
        lambda s: (s if s not in RAW_ABSENT and s in SERVICE_SET else
                   ("other" if s not in RAW_ABSENT else "<absent>"))
    )

    # ── Block 3: State ────────────────────────────────────────────────────────
    conn_state = chunk.get("conn_state", pd.Series(["-"] * n, index=idx)).fillna("-").astype(str).str.strip()
    out["raw_state_v51"] = conn_state.map(STATE_V51_MAP).fillna("OTHER")

    # ── Block 4: Ports ────────────────────────────────────────────────────────
    out["raw_sport"] = _safe_num(chunk.get("id.orig_p", pd.Series(-1, index=idx)), fill=-1).astype(int)
    out["raw_dport"] = _safe_num(chunk.get("id.resp_p", pd.Series(-1, index=idx)), fill=-1).astype(int)

    # ── Block 5: No app-layer columns in IoT-23 conn.log ─────────────────────
    out["dns_qtype"]          = -1
    out["dns_qclass"]         = -1
    out["dns_rcode"]          = -1
    out["raw_http_method"]    = "-"
    out["http_status_code"]   = -1
    out["http_req_body_len"]  = -1
    out["http_resp_body_len"] = -1
    out["raw_ssl_cipher"]     = "-"
    out["raw_ssl_version"]    = "-"
    out["ssl_established"]    = np.int8(0)

    # ── Block 6: Momentum — absent for IoT-23 ─────────────────────────────────
    for col in ["mom_mean","mom_stddev","mom_sum","mom_min","mom_max","mom_rate",
                "mom_srate","mom_drate","mom_TnBPSrcIP","mom_TnBPDstIP",
                "mom_TnP_PSrcIP","mom_TnP_PDstIP","mom_TnP_PerProto","mom_TnP_Per_Dport"]:
        out[col] = -1.0

    # ── Mask bits ─────────────────────────────────────────────────────────────
    masks = compute_mask_bits_raw(
        service_col     = chunk.get("service",    pd.Series(["-"] * n, index=idx)),
        dns_qtype_col   = None,
        http_method_col = None,
        http_status_col = None,
        ssl_cipher_col  = None,
        ssl_version_col = None,
        dport_col       = chunk.get("id.resp_p",  None),
        unsw_col        = None,
        n=n, index=idx,
    )
    for k, v in masks.items():
        out[k] = v

    return out[V51_OUTPUT_COLUMNS]


# ─────────────────────────────────────────────────────────────────────────────
# Bot-IoT Transform
# Source columns: stime, saddr, sport, daddr, dport, proto, state, dur,
#   sbytes, dbytes, spkts, dpkts,
#   mean, stddev, sum, min, max, rate, srate, drate,
#   TnBPSrcIP, TnBPDstIP, TnP_PSrcIP, TnP_PDstIP, TnP_PerProto, TnP_Per_Dport,
#   AR_P_Proto_P_SrcIP, AR_P_Proto_P_DstIP, N_IN_Conn_P_DstIP, N_IN_Conn_P_SrcIP,
#   AR_P_Proto_P_Sport, AR_P_Proto_P_Dport,
#   Pkts_P_State_P_Protocol_P_DestIP, Pkts_P_State_P_Protocol_P_SrcIP,
#   attack, category, subcategory
# ─────────────────────────────────────────────────────────────────────────────

# Argus state → v5.1 mapping
BOTIOT_STATE_MAP = {
    "ACC": "ESTABLISHED", "CON": "ESTABLISHED", "FIN": "ESTABLISHED",
    "CLO": "ESTABLISHED", "ECO": "ESTABLISHED",
    "RST": "RESET",       "TXW": "RESET",
    "URH": "PENDING",     "REQ": "PENDING",     "INT": "PENDING",
    "-":   "OTHER",       "":    "OTHER",        "other": "OTHER",
}

MOM_BOTIOT_MAP = {
    "mom_mean":          "mean",
    "mom_stddev":        "stddev",
    "mom_sum":           "sum",
    "mom_min":           "min",
    "mom_max":           "max",
    "mom_rate":          "rate",
    "mom_srate":         "srate",
    "mom_drate":         "drate",
    "mom_TnBPSrcIP":     "TnBPSrcIP",
    "mom_TnBPDstIP":     "TnBPDstIP",
    "mom_TnP_PSrcIP":    "TnP_PSrcIP",
    "mom_TnP_PDstIP":    "TnP_PDstIP",
    "mom_TnP_PerProto":  "TnP_PerProto",
    "mom_TnP_Per_Dport": "TnP_Per_Dport",
}


def transform_botiot_chunk(chunk: pd.DataFrame) -> pd.DataFrame:
    """Bot-IoT CSV chunk → v5.1 aligned schema (full momentum resolution)."""
    n   = len(chunk)
    idx = chunk.index
    out = _build_output_frame(idx)

    # ── Identity / Metadata ───────────────────────────────────────────────────
    out["dataset_source"] = "botiot"
    out["meta_src_ip"]    = chunk.get("saddr", pd.Series([""] * n, index=idx)).fillna("").astype(str)
    out["meta_dst_ip"]    = chunk.get("daddr", pd.Series([""] * n, index=idx)).fillna("").astype(str)
    out["meta_timestamp"] = _safe_num(chunk.get("stime", pd.Series(0.0, index=idx)), fill=0.0)

    # ── UBT label from category + subcategory ────────────────────────────────
    category   = chunk.get("category",    pd.Series(["Normal"] * n, index=idx)).fillna("Normal").astype(str)
    subcategory= chunk.get("subcategory", pd.Series([""] * n, index=idx)).fillna("").astype(str)
    attack_bin = chunk.get("attack",      pd.Series([0] * n, index=idx)).fillna(0)

    ubt_list = []
    for cat, sub, ab in zip(category, subcategory, attack_bin):
        if str(ab).strip() in {"0", "0.0"}:
            ubt_list.append("NORMAL")
        else:
            # Try subcategory first for finer discrimination, fall back to category
            ubt = map_to_ubt(sub)
            if ubt == "ANOMALY":
                ubt = map_to_ubt(cat)
            ubt_list.append(ubt)

    out["ubt_archetype"]        = ubt_list
    out["univ_specific_attack"] = subcategory.astype(str)

    # ── Block 1: Core ─────────────────────────────────────────────────────────
    out["univ_duration"]  = _safe_num(chunk.get("dur",    pd.Series(0.0, index=idx))).clip(lower=0)
    out["univ_bytes_in"]  = _safe_num(chunk.get("sbytes", pd.Series(0.0, index=idx))).clip(lower=0)
    out["univ_bytes_out"] = _safe_num(chunk.get("dbytes", pd.Series(0.0, index=idx))).clip(lower=0)
    out["univ_pkts_in"]   = _safe_num(chunk.get("spkts",  pd.Series(0.0, index=idx))).clip(lower=0)
    out["univ_pkts_out"]  = _safe_num(chunk.get("dpkts",  pd.Series(0.0, index=idx))).clip(lower=0)

    # ── Block 2: Protocol ─────────────────────────────────────────────────────
    valid_protos = frozenset(PROTO_TOKENS) - {"other"}
    raw_proto = chunk.get("proto", pd.Series(["other"] * n, index=idx)).fillna("other").astype(str).str.lower().str.strip()
    out["raw_proto"]   = raw_proto.apply(lambda p: p if p in valid_protos else "other")
    out["raw_service"] = "<absent>"   # Bot-IoT has no service column

    # ── Block 3: State ────────────────────────────────────────────────────────
    state_raw = chunk.get("state", pd.Series(["INT"] * n, index=idx)).fillna("INT").astype(str).str.strip()
    out["raw_state_v51"] = state_raw.map(BOTIOT_STATE_MAP).fillna("OTHER")

    # ── Block 4: Ports ────────────────────────────────────────────────────────
    out["raw_sport"] = _safe_num(chunk.get("sport", pd.Series(-1, index=idx)), fill=-1).astype(int)
    out["raw_dport"] = _safe_num(chunk.get("dport", pd.Series(-1, index=idx)), fill=-1).astype(int)

    # ── Block 5: No app-layer columns in Bot-IoT ──────────────────────────────
    out["dns_qtype"]          = -1
    out["dns_qclass"]         = -1
    out["dns_rcode"]          = -1
    out["raw_http_method"]    = "-"
    out["http_status_code"]   = -1
    out["http_req_body_len"]  = -1
    out["http_resp_body_len"] = -1
    out["raw_ssl_cipher"]     = "-"
    out["raw_ssl_version"]    = "-"
    out["ssl_established"]    = np.int8(0)

    # ── Block 6: Momentum — ACTUAL 14 UNSW window features ────────────────────
    for out_col, src_col in MOM_BOTIOT_MAP.items():
        if src_col in chunk.columns:
            out[out_col] = _safe_num(chunk[src_col], fill=-1.0)
        else:
            out[out_col] = -1.0

    # ── Mask bits ─────────────────────────────────────────────────────────────
    unsw_col = chunk.get("TnBPSrcIP", None)
    masks = compute_mask_bits_raw(
        service_col     = pd.Series(["-"] * n, index=idx),
        dns_qtype_col   = None,
        http_method_col = None,
        http_status_col = None,
        ssl_cipher_col  = None,
        ssl_version_col = None,
        dport_col       = chunk.get("dport", None),
        unsw_col        = unsw_col,
        n=n, index=idx,
    )
    for k, v in masks.items():
        out[k] = v

    return out[V51_OUTPUT_COLUMNS]




print("✅ Dataset-specific chunk transformers defined:")
print("     ↳ Raw DNA: 14 UNSW momentum window features (all present)")

print("   transform_toniot_chunk() — TON-IoT CSVs")
print("   transform_botiot_chunk() — Bot-IoT CSVs")

print("     ↳ Raw DNA: http_method, ssl_cipher, ssl_version, ssl_established, dns_qtype")
print("     ↳ Raw DNA: detailed-label for UBT mapping (no app-layer cols in conn.log)")
print("   transform_iot23_chunk()  — IoT-23 Zeek conn.log.labeled")

✅ Dataset-specific chunk transformers defined:
     ↳ Raw DNA: 14 UNSW momentum window features (all present)
   transform_toniot_chunk() — TON-IoT CSVs
   transform_botiot_chunk() — Bot-IoT CSVs
     ↳ Raw DNA: http_method, ssl_cipher, ssl_version, ssl_established, dns_qtype
     ↳ Raw DNA: detailed-label for UBT mapping (no app-layer cols in conn.log)
   transform_iot23_chunk()  — IoT-23 Zeek conn.log.labeled


In [42]:

# ─────────────────────────────────────────────────────────────────────────────
# CELL 6 | Partitioned Streaming Loader — IDS Ocean v5.1
#
#  Architecture:
#    • NO ROW CAPS — every row in every source file is processed
#    • pyarrow.parquet.write_to_dataset  → Hive-style partition by ubt_archetype
#    • Output: ocean_v51/ubt_archetype=NORMAL/  .. /SCAN/  .. /DOS_DDOS/  …
#    • IoT-23: zeek_chunk_generator() — #fields-aware, yields chunks
#    • Per-chunk try-except: log failure, skip chunk, continue
#    • Chunk size: 250,000 rows
# ─────────────────────────────────────────────────────────────────────────────

# ── PyArrow partition schema (excludes ubt_archetype — it becomes the dir name)
def _build_pa_schema_partitioned() -> pa.Schema:
    """
    Build pyarrow schema for parquet files inside each partition directory.
    ubt_archetype is EXCLUDED — pyarrow derives it from the directory name.
    """
    EXCLUDE = {"ubt_archetype"}
    str_cols  = {"dataset_source","meta_src_ip","meta_dst_ip",
                 "univ_specific_attack","raw_proto","raw_service","raw_state_v51",
                 "raw_http_method","raw_ssl_cipher","raw_ssl_version"}
    int8_cols = {"ssl_established","has_svc","has_dns","has_http","has_ssl","has_unsw"}
    int64_cols= {"raw_sport","raw_dport","dns_qtype","dns_qclass","dns_rcode",
                 "http_status_code","http_req_body_len","http_resp_body_len"}
    fields = []
    for col in V51_OUTPUT_COLUMNS:
        if col in EXCLUDE:
            continue
        if col in str_cols:
            fields.append(pa.field(col, pa.string()))
        elif col in int8_cols:
            fields.append(pa.field(col, pa.int8()))
        elif col in int64_cols:
            fields.append(pa.field(col, pa.int64()))
        else:
            fields.append(pa.field(col, pa.float64()))
    return pa.schema(fields)


def _df_to_pa_table(df: pd.DataFrame) -> pa.Table:
    """
    Cast DataFrame → Arrow Table enforcing schema types.
    ubt_archetype kept as string (partition key). No schema enforcement
    on the partition column — pyarrow write_to_dataset handles it.
    """
    df = df[V51_OUTPUT_COLUMNS].copy()
    str_cols  = {"dataset_source","meta_src_ip","meta_dst_ip","ubt_archetype",
                 "univ_specific_attack","raw_proto","raw_service","raw_state_v51",
                 "raw_http_method","raw_ssl_cipher","raw_ssl_version"}
    int8_cols = {"ssl_established","has_svc","has_dns","has_http","has_ssl","has_unsw"}
    int64_cols= {"raw_sport","raw_dport","dns_qtype","dns_qclass","dns_rcode",
                 "http_status_code","http_req_body_len","http_resp_body_len"}
    for col in V51_OUTPUT_COLUMNS:
        try:
            if col in str_cols:
                df[col] = df[col].fillna("").astype(str)
            elif col in int8_cols:
                df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype("int8")
            elif col in int64_cols:
                df[col] = pd.to_numeric(df[col], errors="coerce").fillna(-1).astype("int64")
            else:
                df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0.0).astype("float64")
        except Exception:
            pass
    return pa.Table.from_pandas(df, preserve_index=False)


# ── IoT-23 Zeek conn.log Generator (yields DataFrames, no full-file load) ─────
def zeek_chunk_generator(filepath: Path, chunk_size: int = 250_000):
    """
    Generator: parse Zeek ASCII log header (# lines) then yield DataFrame chunks.
      • Reads '#separator' + '#fields' to discover schema
      • pd.read_csv with comment='#' skips all header / meta comment lines
      • Yields DataFrames of at most chunk_size rows
    """
    sep  = "\t"
    cols = None
    try:
        with open(filepath, "r", encoding="utf-8", errors="replace") as fh:
            for raw_line in fh:
                line = raw_line.rstrip("\n")
                if line.startswith("#separator"):
                    parts   = line.split(None, 1)
                    raw_sep = parts[1].strip() if len(parts) > 1 else "\t"
                    sep = (bytes.fromhex(raw_sep[2:]).decode()
                           if raw_sep.startswith("\\x") else raw_sep)
                elif line.startswith("#fields"):
                    fields_str = line[len("#fields"):].lstrip()
                    cols = fields_str.split(sep)
                if cols is not None and not line.startswith("#"):
                    break
    except Exception as e:
        print(f"    ⚠  Header scan failed {filepath.name}: {e}")
        return
    if cols is None:
        print(f"    ⚠  No #fields in {filepath.name} — skipping")
        return
    try:
        reader = pd.read_csv(
            filepath, sep=sep, names=cols, comment="#",
            header=None, low_memory=False,
            encoding="utf-8", on_bad_lines="warn",
            chunksize=chunk_size,
        )
        for chunk in reader:
            # ── Expand compound columns (IoT-23-specific format) ──────────────
            # IoT-23 Zeek headers append 'label' + 'detailed-label' after
            # 'tunnel_parents' using 3-space delimiters instead of tabs.
            # Result: ONE tab-field containing e.g. '-   benign   -'
            # We detect such compound columns by '   ' in the column name and
            # split BOTH the name and the data values into proper sub-columns.
            for col in list(chunk.columns):
                if isinstance(col, str) and '   ' in col:
                    sub_cols = [c.strip() for c in col.split('   ') if c.strip()]
                    expanded = chunk[col].astype(str).str.split(r'\s{2,}', expand=True)
                    for i, sub_col in enumerate(sub_cols):
                        chunk[sub_col] = (
                            expanded.iloc[:, i].str.strip()
                            if i < expanded.shape[1] else '-'
                        )
                    chunk = chunk.drop(columns=[col])
            yield chunk
    except Exception as e:
        print(f"    ⚠  Read error {filepath.name}: {e}")
        return


# ── Main Partitioned Loader ───────────────────────────────────────────────────
def stream_ocean_to_partitioned(ocean_dir: Path, chunk_size: int = 250_000) -> dict:
    """
    Stream ALL raw source datasets → v5.1 transform → Hive-partitioned Parquet.

    Writes to:   ocean_dir/ubt_archetype=NORMAL/  ..SCAN/  ..DOS_DDOS/  ..etc.
    No row caps. Per-chunk try-except.

    Returns:  { 'toniot': N, 'iot23': N, 'botiot': N, 'dropped': N }
    """
    ocean_dir = Path(ocean_dir)

    # Fresh start: wipe any previous partial ocean
    if ocean_dir.exists():
        shutil.rmtree(ocean_dir)
        print(f"🗑  Cleared previous ocean: {ocean_dir.name}/")
    ocean_dir.mkdir(parents=True, exist_ok=True)

    summary = {"toniot": 0, "iot23": 0, "botiot": 0, "dropped": 0}
    write_seq = [0]   # mutable counter for unique filenames

    def _flush(part_df: pd.DataFrame, ds_key: str, label: str) -> int:
        """Transform one chunk + write to partitioned dataset. Returns rows written."""
        if part_df is None or len(part_df) == 0:
            return 0
        try:
            table = _df_to_pa_table(part_df)
            pq.write_to_dataset(
                table,
                root_path=str(ocean_dir),
                partition_cols=["ubt_archetype"],
                basename_template=f"part-{write_seq[0]:07d}-{{i}}.parquet",
            )
            write_seq[0] += 1
            n = len(part_df)
            summary[ds_key] += n
            return n
        except Exception as e:
            summary["dropped"] += 1
            print(f"\n    ⚠  [{label}] chunk dropped: {e}")
            return 0

    t0 = time.time()

    # ─────────────────── [1/3] TON-IoT ───────────────────────────────────────
    print(f"\n{'─'*68}")
    print(f"[1/3] TON-IoT  — {len(toniot_files)} CSVs  (NO CAP — full dataset)")
    print(f"{'─'*68}")
    for fp in toniot_files:
        file_n = 0
        print(f"  ↳ {fp.name}", end="  ", flush=True)
        try:
            for raw_chunk in pd.read_csv(fp, chunksize=chunk_size, low_memory=False,
                                         encoding="utf-8", on_bad_lines="warn"):
                try:
                    part = transform_toniot_chunk(raw_chunk)
                    file_n += _flush(part, "toniot", fp.name)
                except Exception as e:
                    summary["dropped"] += 1
                    print(f"\n    ⚠  chunk transform error ({fp.name}): {e}", end="")
        except Exception as e:
            print(f"\n  ⚠  file read error ({fp.name}): {e}", end="")
        elapsed = time.time() - t0
        print(f"+{file_n:,}  [total toniot={summary['toniot']:,}  elapsed={elapsed:.0f}s]")

    # ─────────────────── [2/3] IoT-23 ────────────────────────────────────────
    print(f"\n{'─'*68}")
    print(f"[2/3] IoT-23   — {len(iot23_logs)} Zeek logs  (NO CAP — full dataset)")
    print(f"{'─'*68}")
    for fp in iot23_logs:
        file_n = 0
        label_p = f"{fp.parent.name}/{fp.name}"
        print(f"  ↳ {label_p}", end="  ", flush=True)
        for raw_chunk in zeek_chunk_generator(fp, chunk_size):
            try:
                part = transform_iot23_chunk(raw_chunk)
                file_n += _flush(part, "iot23", fp.name)
            except Exception as e:
                summary["dropped"] += 1
                print(f"\n    ⚠  chunk transform error ({fp.name}): {e}", end="")
        elapsed = time.time() - t0
        print(f"+{file_n:,}  [total iot23={summary['iot23']:,}  elapsed={elapsed:.0f}s]")

    # ─────────────────── [3/3] Bot-IoT ───────────────────────────────────────
    print(f"\n{'─'*68}")
    print(f"[3/3] Bot-IoT  — {len(botiot_files)} CSVs  (NO CAP — full dataset)")
    print(f"{'─'*68}")
    for fp in botiot_files:
        file_n = 0
        print(f"  ↳ {fp.name}", end="  ", flush=True)
        try:
            for raw_chunk in pd.read_csv(fp, chunksize=chunk_size, low_memory=False,
                                         encoding="utf-8", on_bad_lines="warn"):
                try:
                    part = transform_botiot_chunk(raw_chunk)
                    file_n += _flush(part, "botiot", fp.name)
                except Exception as e:
                    summary["dropped"] += 1
                    print(f"\n    ⚠  chunk transform error ({fp.name}): {e}", end="")
        except Exception as e:
            print(f"\n  ⚠  file read error ({fp.name}): {e}", end="")
        elapsed = time.time() - t0
        print(f"+{file_n:,}  [total botiot={summary['botiot']:,}  elapsed={elapsed:.0f}s]")

    # ── Final summary ─────────────────────────────────────────────────────────
    total_rows = summary["toniot"] + summary["iot23"] + summary["botiot"]
    total_time = time.time() - t0
    size_mb = sum(f.stat().st_size for f in ocean_dir.rglob("*.parquet")) / 1e6

    print(f"\n{'='*68}")
    print(f"✅ IDS Ocean write complete  ({total_time/60:.1f} min)")
    print(f"   Parquet files   : {write_seq[0]}")
    print(f"   Dropped chunks  : {summary['dropped']}")
    print(f"   Ocean size      : {size_mb:.1f} MB")
    print(f"   Rows written    : {total_rows:,}")
    print(f"      toniot        : {summary['toniot']:,}")
    print(f"      iot23         : {summary['iot23']:,}")
    print(f"      botiot        : {summary['botiot']:,}")
    print(f"   Output          : {ocean_dir}")
    print(f"{'='*68}")

    if total_rows == 0:
        raise RuntimeError("❌ No rows written — verify raw data paths in Cell 2")

    return summary


print("📦 IDS Ocean Partitioned Loader ready")
print(f"   stream_ocean_to_partitioned(ocean_dir, chunk_size)")
print(f"   Output structure:")
for arch in UBT_ARCHETYPES:
    print(f"     ocean_v51/ubt_archetype={arch}/part-*.parquet")
print(f"\n   Architecture  : pyarrow.parquet.write_to_dataset (Hive partitioning)")
print(f"   IoT-23 reader : zeek_chunk_generator() — #fields-aware generator")
print(f"   Error policy  : per-chunk try-except → log + skip, never crash")
print(f"   Row caps      : NONE — full IDS Ocean")


📦 IDS Ocean Partitioned Loader ready
   stream_ocean_to_partitioned(ocean_dir, chunk_size)
   Output structure:
     ocean_v51/ubt_archetype=NORMAL/part-*.parquet
     ocean_v51/ubt_archetype=SCAN/part-*.parquet
     ocean_v51/ubt_archetype=DOS_DDOS/part-*.parquet
     ocean_v51/ubt_archetype=BOTNET_C2/part-*.parquet
     ocean_v51/ubt_archetype=EXPLOIT/part-*.parquet
     ocean_v51/ubt_archetype=BRUTE_FORCE/part-*.parquet
     ocean_v51/ubt_archetype=THEFT_EXFIL/part-*.parquet
     ocean_v51/ubt_archetype=ANOMALY/part-*.parquet

   Architecture  : pyarrow.parquet.write_to_dataset (Hive partitioning)
   IoT-23 reader : zeek_chunk_generator() — #fields-aware generator
   Error policy  : per-chunk try-except → log + skip, never crash
   Row caps      : NONE — full IDS Ocean


In [43]:

# ─────────────────────────────────────────────────────────────────────────────
# CELL 7 | Execute Partitioned Ocean Loader
#          Processes ALL rows in ALL source files — NO caps
# ─────────────────────────────────────────────────────────────────────────────
import time as _time

print("=" * 68)
print("  IDS Ocean Loader — Phase 1.1 v5.1")
print(f"  Schema   : {SCHEMA_VERSION}  |  {TOTAL_DIMS} dims")
print(f"  Chunk    : {CHUNK_SIZE:,} rows")
print(f"  Output   : {OCEAN_V51_DIR}")
print(f"  Mode     : FULL DATASET — NO ROW CAPS")
print(f"  Sources  : toniot ({len(toniot_files)} files) | "
      f"iot23 ({len(iot23_logs)} logs) | botiot ({len(botiot_files)} files)")
print("=" * 68)
print()

_t_start = _time.time()

ocean_summary = stream_ocean_to_partitioned(
    ocean_dir  = OCEAN_V51_DIR,
    chunk_size = CHUNK_SIZE,
)

_elapsed = _time.time() - _t_start

print(f"\n📊 Final Ocean Summary:")
total_ocean_rows = 0
for ds in ["toniot", "iot23", "botiot"]:
    rows = ocean_summary[ds]
    total_ocean_rows += rows
    print(f"   {ds:<10}  {rows:>14,} rows")
print(f"   {'TOTAL':<10}  {total_ocean_rows:>14,} rows")
print(f"   Dropped    {ocean_summary['dropped']:>14,} chunks")
print(f"\n   Wall time  : {_elapsed/60:.1f} min")


  IDS Ocean Loader — Phase 1.1 v5.1
  Schema   : v5.1  |  114 dims
  Chunk    : 250,000 rows
  Output   : c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\data\unified\ocean_v51
  Mode     : FULL DATASET — NO ROW CAPS
  Sources  : toniot (23 files) | iot23 (23 logs) | botiot (4 files)

🗑  Cleared previous ocean: ocean_v51/

────────────────────────────────────────────────────────────────────
[1/3] TON-IoT  — 23 CSVs  (NO CAP — full dataset)
────────────────────────────────────────────────────────────────────
  ↳ Network_dataset_1.csv  +1,000,000  [total toniot=1,000,000  elapsed=12s]
  ↳ Network_dataset_10.csv  +1,000,000  [total toniot=2,000,000  elapsed=24s]
  ↳ Network_dataset_11.csv  +1,000,000  [total toniot=3,000,000  elapsed=36s]
  ↳ Network_dataset_12.csv  +1,000,000  [total toniot=4,000,000  elapsed=48s]
  ↳ Network_dataset_13.csv  +1,000,000  [total toniot=5,000,000  elapsed=61s]
  ↳ Network_dataset_14.csv  +1,000,000  [tot

---
## Section 4 — Validation

Verify the aligned schema output: row counts, column completeness, mask distributions, UBT taxonomy.


In [46]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 8 | Partition Validation — IDS Ocean v5.1 + Forensic Evidence Audit
# ─────────────────────────────────────────────────────────────────────────────
import pyarrow.dataset as pad
import pandas as pd

print("=" * 68)
print("   Phase 1.1 v5.1 — IDS Ocean Partition Validation & Audit")
print("=" * 68)

if not OCEAN_V51_DIR.exists():
    raise FileNotFoundError(f"❌ Ocean directory not found: {OCEAN_V51_DIR}")

# ── 1. Partition Statistics ──────────────────────────────────────────────────
partition_dirs = sorted([d for d in OCEAN_V51_DIR.iterdir() if d.is_dir()])
total_parquet_files = list(OCEAN_V51_DIR.rglob("*.parquet"))
total_disk_mb = sum(f.stat().st_size for f in total_parquet_files) / 1e6

print(f"\n📦 Ocean directory  : {OCEAN_V51_DIR}")
print(f"   Partition dirs   : {len(partition_dirs)}")
print(f"   Parquet files    : {len(total_parquet_files):,}")
print(f"   Total disk size  : {total_disk_mb:.1f} MB")

# ── 2. Row Counts per Partition ──────────────────────────────────────────────
print(f"\n📊 Row counts per ubt_archetype partition:")
total_rows_all = 0
partition_stats = {}
for part_dir in partition_dirs:
    arch_name = part_dir.name
    try:
        ds_part = pad.dataset(str(part_dir), format="parquet")
        n_rows = ds_part.count_rows()
    except:
        n_rows = 0
    total_rows_all += n_rows
    partition_stats[arch_name] = n_rows
    # Using your actual total: 351,317,489
    pct = 100 * n_rows / 351317489 
    print(f"   {arch_name:<35} | {n_rows:>14,} rows ({pct:>6.2f}%)")

# ── 3. Forensic Evidence Audit (1M Row Sample) ──────────────────────────────
print(f"\n🔍 Forensic Evidence Audit (Random Sample Across Ocean):")
ds = pad.dataset(str(OCEAN_V51_DIR), format="parquet")
sample_cols = ['has_http', 'has_ssl', 'has_dns', 'has_svc', 'has_unsw']

# CORRECTED SYNTAX: Use .head() to limit the scan
try:
    sample_table = ds.head(1_000_000, columns=sample_cols)
    sample = sample_table.to_pandas()

    for col in sample_cols:
        coverage = sample[col].mean() * 100
        status = "✅" if coverage > 0 else "⚠️  LOW/ZERO"
        print(f"   {status} {col:<10} Coverage: {coverage:>6.2f}%")
except Exception as e:
    print(f"   ❌ Audit failed: {e}")

print(f"\n📐 Virtual Dimension Check: {TOTAL_DIMS} dims ✅ sealed")
print(f"{'='*68}")
print(f" ✅ Phase 1.1 v5.1 IDS Ocean — VALIDATED & AUDITED")

   Phase 1.1 v5.1 — IDS Ocean Partition Validation & Audit

📦 Ocean directory  : c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\data\unified\ocean_v51
   Partition dirs   : 7
   Parquet files    : 4,195
   Total disk size  : 6919.4 MB

📊 Row counts per ubt_archetype partition:
   ubt_archetype=BOTNET_C2             |     61,556,313 rows ( 17.52%)
   ubt_archetype=BRUTE_FORCE           |      1,718,568 rows (  0.49%)
   ubt_archetype=DOS_DDOS              |     32,665,331 rows (  9.30%)
   ubt_archetype=EXPLOIT               |      2,635,460 rows (  0.75%)
   ubt_archetype=NORMAL                |     31,657,548 rows (  9.01%)
   ubt_archetype=SCAN                  |    221,084,172 rows ( 62.93%)
   ubt_archetype=THEFT_EXFIL           |             97 rows (  0.00%)

🔍 Forensic Evidence Audit (Random Sample Across Ocean):
   ✅ has_http   Coverage:  50.81%
   ✅ has_ssl    Coverage:   0.00%
   ✅ has_dns    Coverage:  50.81%
   ✅ has_sv